<a href="https://colab.research.google.com/github/vshalisko/GEE/blob/main/Colab/Landsat_ANN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install pyrsgis

In [35]:
import ee
import geemap
from google.colab import drive
import math, random, glob, time
random.seed(2)
import numpy as np
from pyrsgis import raster

Uso de ANN para clasificacion de Landsat

In [36]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-viacheslavs')

Lectura de capas de datos

In [37]:
punto_interes = ee.Geometry.Point([-103.76, 19.75])
print(punto_interes.getInfo())

L8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(punto_interes).filterDate('2015-03-01', '2015-05-15')
print(L8_collection.size().getInfo())

# Load an image (Landsat 8).
L8_image = L8_collection.first()

# Load builtup 2015 (res 100 m)
ghs_built_image = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2015')
built_2015_lr = ghs_built_image.select('built_surface')

# Load builtup 2015 (res 10 m)
built_2015_hr = ee.Image('DLR/WSF/WSF2015/v1')

# blackBackground = ee.Image(0);


{'type': 'Point', 'coordinates': [-103.76, 19.75]}
4


Visualizar

In [38]:

Map = geemap.Map()
# Center the map and display the image.
Map.centerObject(punto_interes, 8)

# Define the visualization parameters L8
vizParams_L8 = {
  'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
   'min': 5000,
   'max': 15000
  #'gamma': [0.95, 1.1, 1]
}
# Define the visualization parameters Builtup LR
vizBuiltup_LR = {
   'min': 0,
   'max': 1,
  'palette': ['white','black']
}
# Define the visualization parameters Builtup HR
vizBuiltup_HR = {
  min: 0,
  max: 255,
  'palette': ['white','black']
}

# Add layers
Map.addLayer(L8_image, vizParams_L8, 'Composición de falso color', True, 0.5)
Map.addLayer(built_2015_lr, vizBuiltup_LR, 'Builtup 2015 100m', True, 0.5)
Map.addLayer(built_2015_hr, vizBuiltup_HR, 'Builtup 2015 10m', True, 0.5)

# Display the map
Map


Map(center=[19.75, -103.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

Pruebas para convertir a numpy

In [39]:
image_meta = L8_image.getInfo()
imagename = image_meta.get('properties',{}).get('PRODUCT_ID')
print(imagename)

projection = L8_image.projection().getInfo();
print(projection)

#print(geemap.image_props(image).getInfo())

# definición de un region pequeño para el analisis
aoi = ee.Geometry.Polygon(
        [[[-103.39846006557397, 20.721686475573005],
          [-103.39846006557397, 20.694710560279958],
          [-103.36309782192163, 20.694710560279958],
          [-103.36309782192163, 20.721686475573005]]], None, False);

## extraccion de banda en un region pequeño
band_arrs = L8_image.sampleRectangle(region=aoi)
band_4 = band_arrs.get('SR_B4')
band_4_np = np.array(band_4.getInfo())
print(band_4_np.shape)
#band_arr_b4 = band_arrs_np.get('SR_B4')

#print("Multispectral image sample shape: ", band_arr_b4.shape)


None
{'type': 'Projection', 'crs': 'EPSG:32613', 'transform': [30, 0, 552285, 0, -30, 2352915]}
(102, 124)


In [40]:
def loadTiff(in_image, init=None, size_img=None):
    src = raster.read(in_image, bands='all')
    nbands = src.RasterCount
    in_band = src.GetRasterBand(1)  # load one band for size reference
    if init is None:
        xinit,yinit = (0, 0)
    else:
        xinit,yinit = init
    if size_img is None:
        block_xsize, block_ysize = (in_band.XSize, in_band.YSize)
    else:
        block_xsize, block_ysize = size_img

    # read the (multiband) file into an array
    image = src.ReadAsArray(xinit, yinit, block_xsize, block_ysize)
    # reshape from bandsxheightxwidth to wxhxb
    image = np.moveaxis(image, 0, -1)
    return image, block_ysize, block_xsize, nbands

#[img, xsize, ysize, nbands] = loadTiff(os.path.join(drive,image))

In [41]:
#task_config = {
#    'image': image,
#    'fileFormat': 'GeoTIFF',
#    'folder': 'test_L8_image',
#    'fileNamePrefix': imagename[0:19],
#    'crs': projection,
#    'crsTransform': projection.transform,
#    'description': "clipped area",
#    'scale':20,
#    'region':poly_area
#}


# This is how we order it to start
#task = ee.batch.Export.image.toDrive(**task_config )
#task.start()

### check task status - you can see if it failed, it's running or finished
#task.status()

In [42]:
#drive = "/content/drive/MyDrive/exercise/"

### loading the file
#s2_data = "test_L8_image"

### loading and checking
#ds1, bands = raster.read(s2_data)
#print(ds1)
#print(bands.shape)